# cli

> command line interface providing friendly tools

In [1]:
#| default_exp cli

In [2]:
#| export
from python_schematized_config.core import ConfigValidator, load_json, extract_declared_items

import os
import sys
import argparse
import dotenv
from typing import Union
from fastcore.script import *

In [3]:
def validate_env(json_schema: Union[str, dict], dotenv_path: str=None):
    validator = ConfigValidator(json_schema)
    try:
        validator.load_config(dotenv.dotenv_values(dotenv_path))
        return True
    except ConfigValidatorException as ex:
        sys.stderr.write(f'{str(ex)}\n')
        for error in ex.errors:
            sys.stderr.write(f'{error.json_path}:\t{error.message}\n')
        return False

In [4]:
def generate_sample_dotenv(json_schema: Union[str, dict], seed_config: dict=None):
    schema_dict = load_json(json_schema)
    merged_config = dict(os.environ)
    default_dotenv = dotenv.dotenv_values()
    merged_config.update(default_dotenv)
    merged_config.update(seed_config or {})
    extracted_config = extract_declared_items(schema_dict, merged_config)
    out = [
        f'{key}={value}'
        for key, value in extracted_config.items()
    ]
    # for all keys that are in the schema, but NOT in the current config,
    # add them as comments
    for (key, value_schema) in schema_dict['properties'].items():
        if key not in extracted_config:
            out.append(f'# {key}=<{value_schema.get("type")}>')
    return '\n'.join(out)

In [5]:
#| hide
sample_dotenv = generate_sample_dotenv(
    {
        'type': 'object',
        'properties': {
            'STRING': { 'type': 'string' },
            'SOMETHING_ELSE': {},
            'HAS_DEFAULT': { 'type': 'boolean', 'default': 'NO COERCION!' },
        },
    },
    {
        'string_value_with_enum': 'these',
        'MY_INTEGER_VALUE': '1122334',
        'A_NUMERIC_VALUE': '13',
    }
)
print(sample_dotenv)

HAS_DEFAULT=NO COERCION!
# STRING=<string>
# SOMETHING_ELSE=<None>


In [6]:
#| export

@call_parse
def main(
    generate: str = None,              # path to a json schema that validates a dotenv
    schema: str = None,                 # path to json schema used for validation
    validate: Union[bool, str] = None,  # validate a dotenv; if a path not given, assume .env; requires <schema>
):
    "friendly tools to work with schemas and dotenv"
    
    if generate:
        sys.stdout.write(generate_sample_dotenv(generate))
    elif schema and validate:
        if validate is True:
            dotenv_path = '.env'
        else:
            dotenv_path = None
        validate_env(schema, dotenv_path)

In [7]:
#| hide
import nbdev; nbdev.nbdev_export()